#  Create Agents to Research and Write an Article¶


In [1]:
# Install libraries
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

  Using cached typer-0.9.4-py3-none-any.whl.metadata (14 kB)


  Using cached huggingface_hub-0.16.4-py3-none-any.whl.metadata (12 kB)
  Using cached websockets-12.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)


Using cached typer-0.9.4-py3-none-any.whl (45 kB)
Using cached huggingface_hub-0.16.4-py3-none-any.whl (268 kB)
Using cached websockets-12.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (130 kB)
  Attempting uninstall: websockets
    Found existing installation: websockets 11.0.3
    Uninstalling websockets-11.0.3:
      Successfully uninstalled websockets-11.0.3
  Attempting uninstall: typer
    Found existing installation: typer 0.12.3
    Uninstalling typer-0.12.3:
      Successfully uninstalled typer-0.12.3
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.23.3
    Uninstalling huggingface-hub-0.23.3:
      Successfully uninstalled huggingface-hub-0.23.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 4.36.1 requires huggingface-hub>=0.19.3, but you have hug

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Import from crewAI library
from crewai import Agent, Task, Crew

In [6]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [7]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [8]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [9]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [10]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [11]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [12]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [13]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew
**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [16]:
result = crew.kickoff(inputs={"topic": "Hyderabadi Dum Biryani"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Hyderabadi Dum Biryani.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: Hyderabadi Dum Biryani

Outline:
I. Introduction
- Brief history of Hyderabadi Dum Biryani
- Explanation of the cooking technique and ingredients used
II. Key Points
A. Latest Trends
- Rise in popularity of Hyderabadi Dum Biryani in international markets
- Introduction of fusion variations such as tofu or jackfruit biryani
B. Key Players
- Famous restaurants and chefs known for their Hyderabadi Dum Biryani
- Influential food bloggers and social media personalities promoting the dish
C. Noteworthy

I now can give a great answer

Final Answer:
# The Exquisite Flavors of Hyderabadi Dum Biryani

## Introduction

Originating from the royal kitchens of Hyderabad, Hyderabadi Dum Biryani is a culinary masterpiece that has captivated taste buds around the world. This iconic dish combines fragrant basmati rice, tender meat, and a blend of aromatic spices cooked in layers to create a symphony of flavors. The cooking technique involves slow cooking the ingredients in a sealed pot, allowing the flavors to meld together and create a truly unforgettable dining experience.

## Key Points

### Latest Trends

In recent years, Hyderabadi Dum Biryani has seen a surge in popularity in international markets. With its rich and complex flavors, it has become a favorite among food enthusiasts looking to explore the diverse world of Indian cuisine. Moreover, innovative chefs have introduced fusion variations of the classic dish, incorporating ingredients like tofu or jackfruit to cater to a wider audienc

- Display the results of your execution as markdown in the notebook.

In [17]:
from IPython.display import Markdown
Markdown(result)

# The Exquisite Flavors of Hyderabadi Dum Biryani

## Introduction

Originating from the royal kitchens of Hyderabad, Hyderabadi Dum Biryani is a culinary masterpiece that has captivated taste buds around the world. This iconic dish combines fragrant basmati rice, tender meat, and a blend of aromatic spices cooked in layers to create a symphony of flavors. The cooking technique involves slow cooking the ingredients in a sealed pot, allowing the flavors to meld together and create a truly unforgettable dining experience.

## Key Points

### Latest Trends

In recent years, Hyderabadi Dum Biryani has seen a surge in popularity in international markets. With its rich and complex flavors, it has become a favorite among food enthusiasts looking to explore the diverse world of Indian cuisine. Moreover, innovative chefs have introduced fusion variations of the classic dish, incorporating ingredients like tofu or jackfruit to cater to a wider audience.

### Key Players

Several famous restaurants and chefs have made a name for themselves with their exceptional Hyderabadi Dum Biryani offerings. From traditional family-owned eateries to modern culinary establishments, these key players continue to uphold the authenticity and excellence of this beloved dish. Additionally, influential food bloggers and social media personalities have played a significant role in promoting Hyderabadi Dum Biryani and creating a buzz around its timeless appeal.

### Noteworthy News

Hyderabadi Dum Biryani has received numerous accolades and recognition for its exceptional taste and cultural significance. Beyond its delectable flavors, the dish also boasts several health benefits and a high nutritional value, making it a wholesome choice for food lovers seeking both taste and nourishment.

## Target Audience

Whether you're a seasoned food enthusiast, an adventurous explorer of traditional cuisines, or a home cook eager to master authentic recipes, Hyderabadi Dum Biryani offers something for everyone. Its rich history and diverse flavors appeal to a wide range of individuals looking to embark on a culinary journey through the vibrant streets of Hyderabad.

## Call to Action

As you delve into the world of Hyderabadi Dum Biryani, I encourage you to try your hand at making this exquisite dish at home. With a plethora of resources and recipes available online, you can recreate the magic of this royal delicacy in the comfort of your kitchen. Take a leap of faith and savor the tantalizing flavors of Hyderabadi Dum Biryani today!

Remember, the key to a perfect Hyderabadi Dum Biryani lies in the meticulous layering of flavors and the patience to let them simmer to perfection. So, gather your ingredients, follow the traditional cooking techniques, and embark on a culinary adventure that will transport you to the bustling streets of Hyderabad with every bite.